# SAP RAG System - Exemplo de Uso

Este notebook demonstra como usar o sistema RAG para consultar documentos SAP.

## Índice
1. Configuração Inicial
2. Processamento de Documentos
3. Indexação
4. Consultas e Perguntas
5. Análise de Resultados

## 1. Configuração Inicial

Importar as bibliotecas necessárias e configurar o ambiente.

In [ ]:
import sys
from pathlib import Path

# Adicionar o diretório raiz ao path
root_dir = Path.cwd().parent
sys.path.insert(0, str(root_dir))

import logging
logging.basicConfig(level=logging.INFO)

print(f"Diretório raiz: {root_dir}")

In [ ]:
# Importar módulos do sistema RAG
from src.rag_engine import SAPRAGSystem
from src.document_processor import DocumentProcessor
from src.embeddings import EmbeddingsGenerator

print("Módulos importados com sucesso!")

## 2. Inicializar o Sistema RAG

Vamos criar uma instância do sistema RAG com ChromaDB.

In [ ]:
# Inicializar o sistema RAG
rag_system = SAPRAGSystem(
    vector_store_type="chroma",
    chunk_size=1000,
    chunk_overlap=200
)

print("Sistema RAG inicializado!")
print("\nEstatísticas do sistema:")
stats = rag_system.get_system_stats()
print(stats)

## 3. Processar e Indexar Documentos

Vamos processar os documentos SAP do diretório raiz.

In [ ]:
# Diretório com documentos (no diretório raiz do projeto)
documents_path = str(root_dir)

print(f"Processando documentos de: {documents_path}")
print("Isso pode levar alguns minutos...\n")

# Processar e indexar documentos
result = rag_system.ingest_documents(documents_path)

if result['status'] == 'success':
    stats = result['stats']
    print("\n" + "="*60)
    print("DOCUMENTOS INDEXADOS COM SUCESSO!")
    print("="*60)
    print(f"Total de chunks: {stats['total_chunks']}")
    print(f"Total de caracteres: {stats['total_characters']}")
    print(f"Tamanho médio do chunk: {stats['avg_chunk_size']}")
    print(f"Documentos processados: {stats['unique_sources']}")
    print("\nFontes:")
    for source in stats['sources']:
        print(f"  - {Path(source).name}")
else:
    print(f"Erro: {result.get('message')}")

## 4. Fazer Perguntas sobre os Documentos

Agora podemos fazer perguntas em linguagem natural sobre os documentos SAP.

### Pergunta 1: O que é SAP S/4HANA?

In [ ]:
question = "O que é SAP S/4HANA?"

result = rag_system.answer_question(question, top_k=3)

print("="*60)
print(f"PERGUNTA: {question}")
print("="*60)
print("\nRESPOSTA:")
print("-"*60)
print(result['answer'])
print("\n" + "="*60)

### Pergunta 2: Principais módulos do SAP

In [ ]:
question = "Quais são os principais módulos do SAP?"

result = rag_system.answer_question(question, top_k=3)

print("="*60)
print(f"PERGUNTA: {question}")
print("="*60)
print("\nRESPOSTA:")
print("-"*60)
print(result['answer'])
print("\n" + "="*60)

### Pergunta 3: FIORI

In [ ]:
question = "O que é SAP FIORI?"

result = rag_system.answer_question(question, top_k=3)

print("="*60)
print(f"PERGUNTA: {question}")
print("="*60)
print("\nRESPOSTA:")
print("-"*60)
print(result['answer'])
print("\n" + "="*60)

## 5. Análise Detalhada dos Resultados

Vamos analisar em detalhes os resultados de uma consulta.

In [ ]:
question = "Como funciona a integração no SAP?"

result = rag_system.answer_question(question, top_k=5)

print("PERGUNTA:")
print(question)
print("\n" + "="*80)
print("ANÁLISE DETALHADA DAS FONTES")
print("="*80)

for i, source in enumerate(result['sources'], 1):
    print(f"\nFonte {i}:")
    print("-"*80)
    print(f"Arquivo: {source['metadata']['filename']}")
    print(f"Relevância: {source.get('score', 0):.2%}")
    print(f"Chunk: {source['metadata'].get('chunk_index', 'N/A')} de {source['metadata'].get('total_chunks', 'N/A')}")
    print(f"\nConteúdo:")
    print(source['text'][:500] + "..." if len(source['text']) > 500 else source['text'])
    print("-"*80)

## 6. Consulta com Contexto Customizado

Usando a API de baixo nível para obter apenas o contexto relevante.

In [ ]:
question = "Explique tabelas transparentes"

# Usar método query para obter apenas o contexto
query_result = rag_system.query(question, top_k=3)

print("PERGUNTA:", question)
print("\n" + "="*80)
print("CONTEXTO RECUPERADO")
print("="*80)
print(query_result['context'])
print("\n" + "="*80)
print(f"Número de fontes consultadas: {query_result['num_sources']}")

## 7. Teste de Similaridade

Testar a similaridade entre textos usando o gerador de embeddings.

In [ ]:
# Acessar o gerador de embeddings
embeddings_gen = rag_system.embeddings_generator

# Testar similaridade
text1 = "SAP S/4HANA é um sistema ERP"
text2 = "S/4HANA é uma plataforma empresarial da SAP"
text3 = "Gosto de pizza"

sim_12 = embeddings_gen.compute_similarity(text1, text2)
sim_13 = embeddings_gen.compute_similarity(text1, text3)

print("Teste de Similaridade:\n")
print(f"Texto 1: '{text1}'")
print(f"Texto 2: '{text2}'")
print(f"Texto 3: '{text3}'")
print("\nResultados:")
print(f"Similaridade(1, 2): {sim_12:.2%} (textos relacionados)")
print(f"Similaridade(1, 3): {sim_13:.2%} (textos não relacionados)")

## 8. Estatísticas Finais do Sistema

In [ ]:
import json

stats = rag_system.get_system_stats()

print("ESTATÍSTICAS DO SISTEMA RAG")
print("="*80)
print(json.dumps(stats, indent=2, ensure_ascii=False))

## Conclusão

Este notebook demonstrou como:

1. Inicializar o sistema RAG
2. Processar e indexar documentos SAP
3. Fazer perguntas em linguagem natural
4. Analisar resultados e fontes
5. Usar recursos avançados do sistema

### Próximos Passos

- Experimentar com diferentes tamanhos de chunk
- Testar diferentes modelos de embeddings
- Integrar com LLMs para respostas mais elaboradas
- Criar uma interface web com Gradio ou Streamlit